In [57]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# autoreload
%reload_ext autoreload
%autoreload 

In [133]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=64,kernel_size=5, stride=1,requires_sum=True,generator=True):
        super(ConvBlock, self).__init__()
        self.requires_sum = requires_sum
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,stride=stride, padding='same' if generator else 2),
            nn.PReLU() if generator else nn.LeakyReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size,stride=stride, padding='same' if generator else 2) if self.requires_sum else nn.Identity()) # changed in_channels to out_channels 15 Jul 2023 @ 15:39:28
        
    def forward(self, x):
        if self.requires_sum is True:
            out = self.block(x)
            return x + out
        else:
            out = self.block(x)
            return out
        


class ConvBlock1D(nn.Module): # Requires to be rewritten to incooperate the 1DConvolutions
    def __init__(self,in_channels=1, out_channels=64,kernel_size=32, stride=2,padding=2):
        super(ConvBlock1D, self).__init__()
        self.block = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.LeakyReLU()
        )

    def forward(self,x):
        return self.block(x)
    


class Generator(nn.Module):
    def __init__(self,in_channel=2, out_channel=64, blocks=4):
        super(Generator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel,kernel_size=11,stride=1,padding='same'),
            nn.PReLU()
        )

        self.blocks = nn.Sequential(*[ConvBlock() for _ in range(blocks)])
        self.conv = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,kernel_size=5,stride=1,padding='same')

        self.terminal = nn.Sequential(
            ConvBlock(in_channels=out_channel, out_channels=out_channel,requires_sum=False),
            ConvBlock(in_channels=out_channel, out_channels=out_channel,requires_sum=False),
            nn.Conv2d(in_channels=out_channel, out_channels=3,kernel_size=11,stride=1,padding='same')
        )

    def forward(self, z):
        # https://de.wikipedia.org/wiki/Arctan2
        z = self.initial(z)
        out = self.blocks(z)
        out = self.conv(out)
        out = out + z
        out = self.terminal(out)

        mag  = out[:,:1,:,:]
        phase = torch.arctan2(out[:,1:2,:,:], out[:,2:,:,:])
        out = torch.cat([mag, phase],dim=1)
        return out, out.shape 


class Discriminator(nn.Module):
    def __init__(self,in_channels=3,out_channels=32,in_features=2048,out_features=512,in_features_final=2048,blocks=2):
        super(Discriminator, self).__init__()
        self.isft_height = 1
        self.isft_width = 1
        self.batch_size = 4
        
        self.magnitude_path = nn.Sequential(
            ConvBlock(in_channels=1,out_channels=out_channels,stride=2,requires_sum=False,generator=False),
            ConvBlock(in_channels=out_channels,out_channels=out_channels,stride=2,requires_sum=False,generator=False),
            nn.Flatten(1,-1),
            nn.Linear(in_features=32*self.isft_height*self.isft_width ,out_features=32*self.isft_height*self.isft_width),
            nn.LeakyReLU()
            )
        
        self.phase_path = ConvBlock1D()

        # [f(x) if condition else g(x) for x in sequence]
        self.blocks = nn.Sequential(*[ConvBlock1D(in_channels=1, out_channels=64) if blocks == 0 else ConvBlock1D(in_channels=64, out_channels=64) for _ in range(blocks)])

        self.terminal = nn.Sequential(
            nn.Linear(in_features=in_features_final,out_features=1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024,out_features=1)
        )


    def forward(self, comp, magnitude):
        

        phase = torch.istft(comp,n_fft=128)
        phase = self.phase_path(phase)

        self.isft_width = phase.shape[1]
        self.isft_height = phase.shape[2]

        mag = self.magnitude_path(magnitude) # will be 32 * H_mag * W_mag

        # out = torch.cat([mag,phase])
        # out = self.blocks(out)
        # return self.terminal(out)

    @staticmethod
    def convert_to_complex(mag, angle):
        # https://dsp.stackexchange.com/a/72172 
        return mag*(np.cos(angle)+1j* np.sin(angle))

In [117]:
x = torch.randn(128,1,4,4)

gen = Generator(1, 64, 4)
disc = Discriminator(in_features=32*7*7)
tens, shp = gen(x)

In [118]:
from src.data_loader import *
import os


path = '../data/AudioMNIST_Indicies/dummy_labels.csv'
root_dir = '../data/AudioMNIST/'

data = AvianNatureSounds(annotation_file=path,root_dir=root_dir,mode='testing',max_ms=56)
train_loader = DataLoader(dataset=data, batch_size=4, shuffle=True)
batch = next(iter(train_loader)) 

batch[0].shape

torch.Size([4, 2, 29, 29])

In [119]:
import torch

input_tensor = torch.randn((4,3,28,28))

# Assuming you have your 3-channel tensor named 'input_tensor'
# Shape of input_tensor: [4, 3, 28, 28]

# Extract the second and third channels
second_channel = input_tensor[:, 1, :, :]
third_channel = input_tensor[:, 2, :, :]

# Apply arctan2 to the values of the second and third channels
output_tensor = torch.atan2(third_channel, second_channel)

# Create a new tensor by stacking the second and third channels along a new dimension
# output_tensor = torch.stack([second_channel, third_channel], dim=1)

# Shape of output_tensor: [4, 2, 28, 28]


In [120]:
output_tensor.shape

torch.Size([4, 28, 28])

In [173]:
x = torch.randn((4, 3,200,200))

mag  = x[:,:1,:,:]
b = x[:,1:2,:,:]
c = x[:,2:,:,:]

# print(a.shape,b.shape,c.shape)
phase = torch.arctan2(b, c)

# out = torch.cat([mag, phase],dim=1)
# print(out.shape)

def convert_to_complex(mag, angle):
    return mag*(np.cos(angle)+1j* np.sin(angle))

out = convert_to_complex(mag, phase)

bz = out.shape[0]
out_signal = None

for i in range(bz):

    signal = torch.istft(out[i],n_fft=200)

    signal = signal.unsqueeze(1)


    if i == 0:
        out_signal = signal.clone()

    else:
        out_signal = torch.concat([out_signal, signal], dim=0) 




blk = nn.Conv1d(1,32,32,2,padding=16)
blk = ConvBlock1D(1,64,padding=16)
blk_ = ConvBlock1D(64,64,padding=16)
lin = nn.Linear(in_features=out.flatten().shape[0],out_features=1024)
lin_final = nn.Linear(in_features=1024,out_features=1)
rel = nn.LeakyReLU()

blk_(blk_(blk(out_signal))).shape
out = blk_(blk_(blk(out_signal)))

print(out.shape)

out = out.flatten()

print(out.shape)
lin = nn.Linear(in_features=out.flatten().shape[0],out_features=1024)


out = lin(out)
out = rel(out)
val = lin_final(out)

print(val)





torch.Size([4, 64, 157])
torch.Size([40192])
tensor([-0.0187], grad_fn=<AddBackward0>)


In [135]:
32 * 2468

78976